In [1]:
import MDAnalysis as mda
import numpy as np
import os
import shutil
import csv
import networkx as nx
from utils import *
import warnings 

# Suppress warnings specific to MDAnalysis
warnings.filterwarnings("ignore", category=UserWarning, module="MDAnalysis")

# dictionary of charged protein residues with key being the Amber resname and the value being the net charge of that residue
residue_charge_dict = {'ARG':1,'LYS':1,'ASP':-1,'GLU':-1,'MG':2,'ThDP':-3}

c:\Users\gbonn\anaconda3\envs\md-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The purpose of this script is to read in the end point of an int1 to DMC (Donor Michaelis-Complex) scan and remake the MM parameters for the unbound donor. This way we can follow the scan with an MM minimization where the donor can move and the ThDP is kept fixed. This script will identify what atoms in INI (intermediate 1) are the donor and what atoms are ThDP. It will then rename the residues and call a bash script to run the AMBER commands to get the new set of parameters. It will also create the MM optimization script

In [8]:
head_dir = 'temp_structures/'
curr_substrate= '6'

substrate_base_charge = -1  
starting_intermediate = 'INI'
ending_intermediate = 'DON'

working_dir = head_dir 
# read in endpoint of the scan 
donor_substrate = curr_substrate
complex = mda.Universe(working_dir + '/tmp.pdb')
QM_resids = [1113,1114]
MM_resids = [30,31,32,33,34,78,79,80,82,107,117,118,175,946,947,948,950,951,969,971,972,974,999,1000,1003,1028,1030,1031,1034,1045,1113,1114,1225,1919,2292,2529,2894,3100,3475,3645,3670,3953,4130,4203,4248,4262,4328,4363,4548,5237,5303,5351,5422] #read_resids_from_csv(working_dir + 'prep/QM_2_and_10A_Active_residues.csv') 

# Get the charge of the QM and MM systems 
if curr_substrate in ['4','6','11','16']:
    additional_charge = -1
else:
    additional_charge = 0

residue_charge_dict[ending_intermediate] = substrate_base_charge + additional_charge

# get the complex atoms 
ini = complex.select_atoms("resname " + starting_intermediate)

# guess bonds based on vDW distances (this is not recommended)
ini_bond_connectivity = mda.topology.core.guess_bonds(ini.atoms,ini.positions)

# Create a graph where nodes are atom indices and edges are bonds
ini_graph = nx.Graph()
ini_graph.add_edges_from(ini_bond_connectivity)

# Find connected components
connected_components = list(nx.connected_components(ini_graph))



41


((17268, 17269), (17268, 17277), (17268, 17279), (17269, 17270), (17269, 17280), (17270, 17271), (17270, 17272), (17270, 17273), (17271, 17274), (17271, 17275), (17271, 17276), (17274, 17278), (17274, 17281), (17282, 17283), (17282, 17286), (17283, 17284), (17284, 17285), (17284, 17305), (17285, 17286), (17285, 17307), (17286, 17287), (17287, 17288), (17287, 17308), (17287, 17309), (17288, 17289), (17288, 17310), (17288, 17311), (17289, 17290), (17290, 17293), (17290, 17296), (17290, 17299), (17291, 17294), (17291, 17297), (17291, 17299), (17291, 17300), (17292, 17295), (17292, 17304), (17295, 17298), (17295, 17306), (17298, 17301), (17301, 17302), (17301, 17303), (17302, 17312), (17302, 17313), (17303, 17304), (17303, 17305), (17304, 17314), (17305, 17315), (17305, 17316), (17306, 17317), (17306, 17318), (17306, 17319), (17307, 17320), (17307, 17321), (17307, 17322))


In [6]:
# Create a graph where nodes are atom indices and edges are bonds
G = nx.Graph()
G.add_edges_from(ini_bond_connectivity)

# Find connected components
connected_components = list(nx.connected_components(G))
print(len(connected_components[0]))

14


In [ ]:

# Print results
print(f"Number of connected segments: {len(connected_components)}")

# Print each segment's atoms
for i, component in enumerate(connected_components):
    atoms = [ini.atoms[idx].name for idx in component]
    print(f"Segment {i+1}: {atoms}")

# Check if residue is fully connected
if len(connected_components) > 1:
    print("Warning: The residue has multiple disconnected segments!")
else:
    print("The residue is fully connected.")

In [ ]:

receptor = complex.select_atoms("not resname " + starting_intermediate)
atom_dict = get_ini_indexes(ini)


# get the index of these atoms in the context of the complex universe
C1_index = ini.atoms[atom_dict['C1']].index
C2_index = ini.atoms[atom_dict['C2']].index
C3_index = ini.atoms[atom_dict['C3']].index
O1_index = ini.atoms[atom_dict['O1']].index
O2_index = ini.atoms[atom_dict['CO2_0']].index
O3_index = ini.atoms[atom_dict['CO2_1']].index

'''    
# guess the position of the hydrogen atom that we want to add
C1_coords = complex.atoms[C1_index].position
C2_coords = complex.atoms[C2_index].position
O1_coords = complex.atoms[O1_index].position
vector_C1_to_C2 = C2_coords - C1_coords
unit_vector = vector_C1_to_C2 / np.linalg.norm(vector_C1_to_C2)
guess_H1_coords = O1_coords + unit_vector * bond_dists['O-H'] # C2 should be located 1.54 A away from C1

# creat a new universe for H
H_u = mda.Universe.empty(1,
                            n_residues=1,
                            trajectory=True) # necessary for adding coordinates
H_count = np.count_nonzero(ini.atoms.types == 'H')
H_u.add_TopologyAttr('name',['H'+str(H_count)])
H_u.add_TopologyAttr('type', ['H'])
H_u.add_TopologyAttr('resname', [starting_intermediate])
H_u.add_TopologyAttr('resid', [ini.resids[0]])
H_u.atoms.positions = [guess_H1_coords]
'''

indices_to_remove = [C3_index,O2_index,O3_index]
# Select atoms to keep (all atoms excluding the ones in indices_to_remove)
mask = ~np.isin(ini.atoms.indices, indices_to_remove)
atoms_to_keep = ini.atoms[mask]
modified_ini_universe = mda.Merge(atoms_to_keep)

# write a pdb for just the INO molecule
output_dir = working_dir + 'int2/prep/' 

#ino_universe = mda.Merge(modified_ini_universe.atoms,H_u.atoms)

ino_universe = modified_ini_universe
for atom in ino_universe.atoms:
    atom.residue.resid = ini.resids[0]
    atom.residue.resname = ending_intermediate

write_universe(output_dir,'ino.pdb',ino_universe)

# add the substrate and ThDP to a single universe
ino_complex = mda.Merge(receptor.atoms,ino_universe.atoms)

write_universe(output_dir,'ino_complex.pdb',ino_complex)
edit_protein_files(output_dir,'ino_complex.pdb')

# Run the bash script from the specified directory

result = subprocess.run(
    ["bash", "/expanse/lustre/projects/nwu118/gbonnanzio/Generate_QMMM_Geometries/scripts/assemble_int2.sh", str(residue_charge_dict[ending_intermediate]), output_dir],
    cwd=head_dir,
    capture_output=True,
    text=True
)

if result.returncode != 0:
    print('ERROR WITH tleap')
    print(result.stderr)  # Print the error output
else:
    print('tleap was successful')
    #print(result.stdout)  # Print the normal output


# read back in the amber generated structure that will fix resids 
mm_output_dir = working_dir + 'int2/mm_opt/'
ino_complex = mda.Universe(mm_output_dir + 'qm_complex.pdb')

# get charge of MM region 
MM_charge = 0
for residue in ino_complex.residues:
    curr_resname = residue.resname
    if curr_resname in residue_charge_dict:
        MM_charge += residue_charge_dict[curr_resname]
        
print('Charge of entire system:',MM_charge)

# get the MM atom indexes
MM_atom_indexes = get_atoms_by_reslist(ino_complex,MM_resids)
# simplify lists to write to file
MM_list = simplify_integer_list(MM_atom_indexes)

# output the MM script
write_MM_orca_script(MM_list,MM_charge,mm_output_dir,'mm_opt.inp')
shutil.copy('scripts/template_orca_job_expanse.sh',mm_output_dir + 'orca_job_expanse.sh')

# calculate the charge of our system
total_QM_charge = 0
# get charge of QM region 
for residue in QM_resids:
    print(residue)
    resname = ino_complex.residues[residue-1].resname
    print(resname)
    if resname in residue_charge_dict:
        total_QM_charge += residue_charge_dict[resname]
    
QM_atoms = ino_complex.select_atoms('resid ' + ' or resid '.join([str(i) for i in QM_resids]))
QM_residues = set([i.residue for i in QM_atoms])
active_atoms = ino_complex.select_atoms('resid ' + ' or resid '.join([str(i) for i in active_resids]))
active_residues = set([i.residue for i in active_atoms])
QM_atoms_indexes = get_atoms_by_reslist(ino_complex,QM_resids)
active_atoms_indexes = get_atoms_by_reslist(ino_complex,active_resids)

QM_list = simplify_integer_list(QM_atoms_indexes)
active_list = simplify_integer_list(active_atoms_indexes)

fixed_atoms = extract_fixed_atoms(working_dir + '/opt/opt.out')
threshold_index = ini_max_index - 3
fixed_atoms_modified = []
for fixed_atom in fixed_atoms:
    if fixed_atom > threshold_index:
        fixed_atoms_modified.append(fixed_atom-3)
    else:
        fixed_atoms_modified.append(fixed_atom)
    
fixed_atoms_list = simplify_integer_list(fixed_atoms_modified)

qm_output_dir = working_dir + 'int2/opt/'
write_QMMM_orca_script(QM_list,active_list,total_QM_charge,qm_output_dir,'opt.inp',fixed_atoms_list)